In [0]:
import numpy as np
import os
import cv2
import random
import imutils
import matplotlib.pyplot as plt
from tqdm import tqdm

TRAIN_DIR='D://Personal Files/Python Programs/DeepLearningSentdex/all/train'
TEST_DIR='D://Personal Files/Python Programs/DeepLearningSentdex/all/test'
IMG_SIZE=50
def label_name(img):
    label=img.split('.')[-3]
    if(label=='cat'):
        return 1
    elif(label=='dog'):
        return 0

def create_train_data():
    training_data=[]
    for img in tqdm(os.listdir(TRAIN_DIR)):
        try:
            label=label_name(img)
            path=os.path.join(TRAIN_DIR,img)
            image=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            image=cv2.resize(image,(IMG_SIZE,IMG_SIZE))
            training_data.append([np.array(image),label])
        except Exception as e:
            pass
    random.shuffle(training_data)
    np.save('D://Personal Files/Python Programs/DeepLearningSentdex/training_data.npy',training_data)
    
create_train_data()

def create_testing_data():
    testing_data=[]
    for img in tqdm(os.listdir(TEST_DIR)):
        try:
            label=img.split('.')[0]
            path=os.path.join(TEST_DIR,img)
            image=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            image=cv2.resize(image,(IMG_SIZE,IMG_SIZE))
            testing_data.append([image])
        except Exception as e:
            pass
    testing_data=np.array(testing_data)
    np.save('D://Personal Files/Python Programs/DeepLearningSentdex/testing_data.npy',testing_data)

create_testing_data()

    


In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import TensorBoard
import time

train=np.load('D://Personal Files/Python Programs/DeepLearningSentdex/training_data.npy')

NAME="CatsvsDogscnn_{}".format(int(time.time()))
tensorboard=TensorBoard(log_dir="D://Personal Files/Python Programs/DeepLearningSentdex/logs/{}".format(NAME))

IMG_SIZE=50
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

model=Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(256,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X,Y,batch_size=100,epochs=1,validation_split=0.1,callbacks=[tensorboard])

model.save('D://Personal Files/Python Programs/DeepLearningSentdex/DogsvsCats.model')

In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import TensorBoard
import time

train=np.load('D://Personal Files/Python Programs/DeepLearningSentdex/training_data.npy')
IMG_SIZE=50
X=np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y=[i[1] for i in train]
X=X/255.0
dense_layers=[0,1,2]
layer_sizes=[32,64,128,512]
conv_layers=[1,2,3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME="{}_conv_{}_nodes_{}_dense_{}".format(conv_layer,layer_size,dense_layer,int(time.time()))
            print(NAME)
            tensorboard=TensorBoard(log_dir="D://Personal Files/Python Programs/DeepLearningSentdex/logs/{}".format(NAME))
            model=Sequential()
            model.add(Conv2D(layer_size,(3,3),input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2,2)))

            for _ in range(conv_layer-1):
                model.add(Conv2D(layer_size,(3,3),input_shape=X.shape[1:]))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2,2)))

            model.add(Flatten())
            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))
                model.add(Dropout(0.1))
            model.add(Dense(1))
            model.add(Activation('sigmoid'))
            model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
            model.fit(X,Y,batch_size=32,epochs=3,validation_split=0.2,callbacks=[tensorboard])
            
